In [13]:
import numpy as np
from lab1.Lab1_FK_answers import *
from scipy.spatial.transform import Rotation as R

In [ ]:
bvh = open("../data/walk60.bvh", 'r')

In [ ]:
line = bvh.readline()
while line != "MOTION\n":
    line = bvh.readline()

In [ ]:
line = bvh.readline()

In [ ]:
line

In [ ]:
line.split()

In [ ]:
len(line.split())

In [ ]:
306 - 125 + 2

In [ ]:
bvh_path = "../data/walk60.bvh"

In [ ]:
load_motion_data(bvh_path)[0, :]

In [3]:
a = np.arange(9)
b = a.reshape(-1, 3)

In [6]:
b

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [8]:
b[1:][0, 0] = 999

In [9]:
a

array([  0,   1,   2, 999,   4,   5,   6,   7,   8])

In [10]:
b, b[1:], b[1:, :]

(array([[  0,   1,   2],
        [999,   4,   5],
        [  6,   7,   8]]),
 array([[999,   4,   5],
        [  6,   7,   8]]),
 array([[999,   4,   5],
        [  6,   7,   8]]))

In [12]:
np.array_equal(b[1:], b[1:, :])

True

In [14]:
r = R.from_quat([1, 0, 0, 0])

In [20]:
r

In [19]:
r.as_quat()

array([1., 0., 0., 0.])

In [ ]:
def pose_joint_orientations(joint_parents: List[int],
                            pose: np.ndarray) -> List[R]:
    rotations = pose.reshape(-1, 3)[1:]
    orientations = [R.from_euler("XYZ", rotations[0], degrees=True)]
    
    for i in range(1, len(joint_parents)):
        orientations.append(orientations[joint_parents[i]] * R.from_euler("XYZ", rotations[i], degrees=True))
    
    return orientations

In [ ]:
def pose_joint_positions(joint_parents: List[int], 
                         joint_offsets: np.ndarray, 
                         joint_orientations: List[R]) -> List[np.ndarray]:
    positions = [joint_offsets[0]]
    
    for i in range(1, len(joint_parents)):
        positions[joint_parents[i]] + joint_orientations[i].apply(joint_offsets[i])
    
    
    return positions

In [ ]:
def part2_forward_kinematics(joint_names, joint_parents, joint_offsets, motion_data, frame_id):
    """请填写以下内容
    输入: part1 获得的关节名字，父节点列表，偏移量列表
        motion_data: np.ndarray，形状为(N,X)的numpy数组，其中N为帧数，X为Channel数
        frame_id: int，需要返回的帧的索引
    输出:
        joint_positions: np.ndarray，形状为(M, 3)的numpy数组，包含着所有关节的全局位置
        joint_orientations: np.ndarray，形状为(M, 4)的numpy数组，包含着所有关节的全局旋转(四元数)
    Tips:
        1. joint_orientations的四元数顺序为(x, y, z, w)
        2. from_euler时注意使用大写的XYZ
    """
    joint_orientations = pose_joint_orientations(joint_parents, motion_data[frame_id])
    return (np.stack(pose_joint_positions(joint_parents, joint_offsets, joint_orientations), axis=0, dtype=np.float64), 
            np.stack([o.as_quat() for o in joint_orientations], axis=0, dtype=np.float64))